This code calculates and stores cosmological power spectra 
and related coefficients for weak lensing and matter power spectra based on cosmological models.

In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '../LimberCloud/' #'/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

Choosing a cosmological model.

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY_INFO = json.load(file)

COSMOLOGY = pyccl.Cosmology(
    h = COSMOLOGY_INFO['H'],
    w0 = COSMOLOGY_INFO['W0'],
    wa = COSMOLOGY_INFO['WA'], 
    n_s = COSMOLOGY_INFO['NS'], 
    A_s = COSMOLOGY_INFO['AS'],
    m_nu = COSMOLOGY_INFO['M_NU'],  
    Neff = COSMOLOGY_INFO['N_EFF'],
    Omega_b = COSMOLOGY_INFO['OMEGA_B'], 
    Omega_k = COSMOLOGY_INFO['OMEGA_K'], 
    Omega_c = COSMOLOGY_INFO['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

Calculate the power spectrum.

In [ ]:
# Define the redshift grid
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

# Calculate corresponding scale factors and comoving distances
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Calculate the power spectrum on the grid
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=COSMOLOGY, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

In [5]:
# Numerical integral

def integral_I5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        result = ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1)) ** 2
        
        result = result * (((chi / chi2) ** 3) * power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    def formula_n(chi):
        result = ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1)) ** 2
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

In [ ]:
# Coefficient

def coefficient_J5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (1 / (10080 * b ** 2)) * (b ** 2 * (1785 + z * ( - 562 + 79 * z) + 42 * b ** 2 * (15 + ( - 6 + z) * z) + 6 * b * (350 + z * ( - 124 + 19 * z))) - 
8 * b * (1 + b) * (432 + z * ( - 129 + 17 * z) + 12 * b * (21 + ( - 7 + z) * z)) * numpy.log(1 + b) + 60 * (1 + b) ** 2 * (28 + ( - 8 + z) * z) * numpy.log(1 + b) ** 2)
    else:
        formula = (1 / (5292000 * a ** 5 * b ** 2)) * (a * b ** 2 * (245 * a ** 2 * (420 * p + a * ( - 600 + a ** 4 * (4030 - 2439 * p) - 90 * (39 + 40 * b) * p + 5 * a ** 2 * ( - 220 - 60 * b * ( - 2 * ( - 9 + p) + 9 * b * ( - 2 + p)) + 133 * p) + 
a ** 3 * ( - 2150 + 300 * b * (30 - 17 * p) + 404 * p) + 20 * a * (495 + 34 * p + 90 * b * (6 + p)))) + 
14 * a * ( - 10920 * p - 70 * a ** 3 * ( - 340 + 150 * b * ( - 6 + p) + 163 * p) + a ** 6 * ( - 85365 + 60096 * p) + 420 * a * (35 + 3 * (53 + 50 * b) * p) - 
70 * a ** 2 * (1755 + 232 * p + 450 * b * (4 + p)) + 14 * a ** 5 * (1010 - 303 * p + 75 * b * ( - 170 + 117 * p)) + 
7 * a ** 4 * (3325 - 952 * p + 750 * b * (4 - p + 6 * b * ( - 3 + 2 * p)))) * z + 
(a ** 7 * (420672 - 322375 * p) + 59220 * p + 35 * a ** 3 * ( - 3248 + 1260 * b * ( - 5 + p) + 1371 * p) + a ** 6 * ( - 29694 + 4410 * b * (195 - 148 * p) + 11624 * p) - 
210 * a * (364 + 3 * (481 + 420 * b) * p) + 14 * a ** 4 * ( - 5705 + 1941 * p + 525 * b * ( - 10 + 3 * p)) + 420 * a ** 2 * (1113 + 197 * p + 105 * b * (10 + 3 * p)) - 
7 * a ** 5 * (6664 - 2441 * p + 210 * b * (25 - 9 * p + 75 * b * ( - 4 + 3 * p)))) * z ** 2) + 
420 * ( - 210 * ( - 1 + a) ** 5 * b ** 2 * (5 * a ** 2 * (p + a * ( - 4 + 3 * p)) - 2 * a * (2 * p + a * ( - 5 + 6 * p + 3 * a * ( - 5 + 4 * p))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a ** 2) + p + a * (3 + 2 * a * (3 + 5 * a)) * p) * z ** 2) * numpy.log(1 - a) ** 2 + ( - 1 + a) ** 3 * b * numpy.log(1 - a) * (b * (35 * a ** 2 * ( - 10 * a * ( - 1 + a * (2 + 17 * a + 18 * b)) + ( - 7 + a * (11 + 60 * b + a * (59 + 117 * a + 120 * b))) * p) - 
14 * a * ( - 5 * a * ( - 7 + a * (11 + 60 * b + a * (59 + 117 * a + 120 * b))) + 2 * ( - 13 + a * (17 + 75 * b + a * (77 + 150 * b + 3 * a * (49 + 74 * a + 75 * b)))) * p) * z + 
( - 14 * a * ( - 13 + a * (17 + 75 * b + a * (77 + 150 * b + 3 * a * (49 + 74 * a + 75 * b)))) + 
( - 141 + a * (159 + 669 * a + 1249 * a ** 2 + 1864 * a ** 3 + 2500 * a ** 4 + 630 * (1 + a * (2 + a * (3 + 4 * a))) * b)) * p) * z ** 2) + 
420 * ( - 1 + a) * a * (1 + b) * (5 * a ** 2 * (p + a * ( - 4 + 3 * p)) - 2 * a * (2 * p + a * ( - 5 + 6 * p + 3 * a * ( - 5 + 4 * p))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a ** 2) + p + a * (3 + 2 * a * (3 + 5 * a)) * p) * z ** 2) * numpy.log(1 + b)) - 
a ** 2 * (1 + b) * numpy.log(1 + b) * (b * ( - 420 * p * z ** 2 + 210 * a * z * (4 * z + p * (8 + z)) + a ** 6 * ( - 5950 + 4095 * p + 42 * (195 - 74 * z) * z + 4 * p * z * ( - 1554 + 625 * z)) + 
70 * a ** 2 * ( - 6 * z * (10 + z) + p * ( - 30 + ( - 12 + z) * z)) + 35 * a ** 3 * (240 - 4 * ( - 15 + z) * z + p * (30 + ( - 8 + z) * z)) + 
7 * a ** 5 * ( - 25 * ( - 64 + 35 * p + 70 * z) + 2 * z * (p * (594 - 224 * z) + 297 * z) + 30 * b * ( - 30 + 20 * p + 5 * (8 - 3 * z) * z + 6 * p * z * ( - 5 + 2 * z))) + 
7 * a ** 4 * (50 * ( - 12 + p) - 20 * ( - 5 + p) * z + ( - 10 + 3 * p) * z ** 2 - 150 * b * ( - 4 * (3 + ( - 3 + z) * z) + p * (6 + z * ( - 8 + 3 * z))))) + 
210 * a ** 4 * (1 + b) * ( - 20 * (3 + ( - 3 + a) * a) + 5 * (6 + a * ( - 8 + 3 * a)) * p + 2 * (30 + 5 * a * ( - 8 + 3 * a) - 20 * p + 6 * (5 - 2 * a) * a * p) * z + 
(5 * ( - 4 + 3 * p) + 2 * a * (15 - 12 * p + a * ( - 6 + 5 * p))) * z ** 2) * numpy.log(1 + b))))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123 4940.716645699535 4957.130588921395 11370.147631619064 11268.969970366385 1.75 1.76


In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

1504.7440545173968 1494.1532988879824 -0.007038243877834138


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 88.58943766284152 0.0 1826.8838600584127 0.0 0.01


In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

2511343.4578193235 2511343.4578193245 4.440892098500626e-16
